- Обучите CNN (самописная) на CIFAR-100.

In [1]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets, models
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchsummary import summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
dataset = datasets.CIFAR100(root='/media/dmitriy/Disk/Downloads/ai_torch_for_nn_data/', train=True, download=True)

def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.2, random_state=42)
    return X_train, X_test

class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

100%|██████████| 169001437/169001437 [00:49<00:00, 3425114.70it/s]


Extracting /media/dmitriy/Disk/Downloads/ai_torch_for_nn_data/cifar-100-python.tar.gz to /media/dmitriy/Disk/Downloads/ai_torch_for_nn_data/


In [4]:
trans_actions = transforms.Compose([
                                    transforms.RandomGrayscale(0.2),
                                    transforms.ColorJitter(brightness=.3, hue=.4),
                                    transforms.RandomRotation(degrees=(0,5)),
                                    transforms.ToTensor()])

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=2)

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.dp_one = nn.Dropout(0.2)
        self.dp_two = nn.Dropout(0.25)
        
        self.bn_one = torch.nn.BatchNorm2d(3)
        self.conv_one = torch.nn.Conv2d(3, 60, 3, padding=1)
        self.bn_two = torch.nn.BatchNorm2d(60)
        self.conv_two = torch.nn.Conv2d(60, 120, 3, padding=1)
        self.bn_three = torch.nn.BatchNorm2d(120)
        self.conv_three = torch.nn.Conv2d(120, 240, 3, padding=1)
        self.bn_four = torch.nn.BatchNorm2d(240)
        self.fc1 = torch.nn.Linear(3840, 1024)
        self.fc2 = torch.nn.Linear(1024, 256)
        self.out = torch.nn.Linear(256, 100)
        
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_one(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_two(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return self.out(x)
       
net = Net()

In [7]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [8]:
summary(net.to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1            [-1, 3, 32, 32]               6
            Conv2d-2           [-1, 60, 32, 32]           1,680
       BatchNorm2d-3           [-1, 60, 16, 16]             120
            Conv2d-4          [-1, 120, 16, 16]          64,920
       BatchNorm2d-5            [-1, 120, 8, 8]             240
            Conv2d-6            [-1, 240, 8, 8]         259,440
       BatchNorm2d-7            [-1, 240, 4, 4]             480
           Dropout-8                 [-1, 3840]               0
            Linear-9                 [-1, 1024]       3,933,184
          Dropout-10                 [-1, 1024]               0
           Linear-11                  [-1, 256]         262,400
           Linear-12                  [-1, 100]          25,700
Total params: 4,548,170
Trainable params: 4,548,170
Non-trainable params: 0
---------------------------

In [9]:
num_epochs = 10
net.train()
for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        if (i != 0) & (i % 100 == 0):    # печатаем каждые 100 mini-batches
            net.eval() 
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Train acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        net.train()  
print('Training is finished!')

Epoch [1/7]. Step [101/313]. Loss: 0.033. Train acc: 0.059. Test acc: 0.114
Epoch [1/7]. Step [201/313]. Loss: 0.030. Train acc: 0.124. Test acc: 0.148
Epoch [1/7]. Step [301/313]. Loss: 0.028. Train acc: 0.163. Test acc: 0.203
Epoch [2/7]. Step [101/313]. Loss: 0.026. Train acc: 0.215. Test acc: 0.237
Epoch [2/7]. Step [201/313]. Loss: 0.024. Train acc: 0.243. Test acc: 0.281
Epoch [2/7]. Step [301/313]. Loss: 0.023. Train acc: 0.269. Test acc: 0.290
Epoch [3/7]. Step [101/313]. Loss: 0.022. Train acc: 0.310. Test acc: 0.323
Epoch [3/7]. Step [201/313]. Loss: 0.021. Train acc: 0.322. Test acc: 0.330
Epoch [3/7]. Step [301/313]. Loss: 0.021. Train acc: 0.329. Test acc: 0.352
Epoch [4/7]. Step [101/313]. Loss: 0.019. Train acc: 0.371. Test acc: 0.367
Epoch [4/7]. Step [201/313]. Loss: 0.019. Train acc: 0.379. Test acc: 0.388
Epoch [4/7]. Step [301/313]. Loss: 0.019. Train acc: 0.384. Test acc: 0.394
Epoch [5/7]. Step [101/313]. Loss: 0.017. Train acc: 0.442. Test acc: 0.392
Epoch [5/7].

- Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

In [10]:
resnet50 = models.resnet50(pretrained=True)

/home/dmitriy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dmitriy/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/dmitriy/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [04:20<00:00, 393kB/s]


In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


set_parameter_requires_grad(resnet50, True)
resnet50.fc = nn.Linear(2048, 100)
summary(resnet50.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [12]:
resnet50 = resnet50.to(device)

In [13]:
train_actions = transforms.Compose([transforms.Resize(256),
                                    transforms.RandomCrop(224, padding=4), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

In [14]:
train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, train_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=2)

In [16]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [17]:
num_epochs = 10
resnet50.train()
for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        if (i != 0) & (i % 100 == 0):    # печатаем каждые 100 mini-batches
            resnet50.eval()
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Train acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        resnet50.train()
print('Training is finished!')

Epoch [1/7]. Step [101/313]. Loss: 0.027. Train acc: 0.257. Test acc: 0.428
Epoch [1/7]. Step [201/313]. Loss: 0.018. Train acc: 0.441. Test acc: 0.482
Epoch [1/7]. Step [301/313]. Loss: 0.016. Train acc: 0.481. Test acc: 0.519
Epoch [2/7]. Step [101/313]. Loss: 0.014. Train acc: 0.522. Test acc: 0.539
Epoch [2/7]. Step [201/313]. Loss: 0.014. Train acc: 0.531. Test acc: 0.548
Epoch [2/7]. Step [301/313]. Loss: 0.013. Train acc: 0.540. Test acc: 0.553
Epoch [3/7]. Step [101/313]. Loss: 0.013. Train acc: 0.556. Test acc: 0.563
Epoch [3/7]. Step [201/313]. Loss: 0.013. Train acc: 0.561. Test acc: 0.572
Epoch [3/7]. Step [301/313]. Loss: 0.012. Train acc: 0.563. Test acc: 0.567
Epoch [4/7]. Step [101/313]. Loss: 0.012. Train acc: 0.578. Test acc: 0.575
Epoch [4/7]. Step [201/313]. Loss: 0.012. Train acc: 0.576. Test acc: 0.576
Epoch [4/7]. Step [301/313]. Loss: 0.012. Train acc: 0.568. Test acc: 0.584
Epoch [5/7]. Step [101/313]. Loss: 0.011. Train acc: 0.594. Test acc: 0.581
Epoch [5/7].

- Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

In [18]:
resnet50 = models.resnet50(pretrained=True)
set_parameter_requires_grad(resnet50, True)
resnet50.fc = nn.Linear(2048, 100)
resnet50 = resnet50.to(device)

In [19]:
train_actions = transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.RandomGrayscale(0.2),
                                    transforms.ColorJitter(brightness=.3, hue=.4),
                                    transforms.RandomRotation(degrees=(0,5)),
                                    transforms.RandomCrop(224, padding=4), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])


train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, train_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

In [20]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128,
                                           shuffle=True,
                                           num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=128,
                                           shuffle=False,
                                           num_workers=2)

In [21]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [22]:
num_epochs = 10
resnet50.train()
for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        if (i != 0) & (i % 100 == 0):    # печатаем каждые 100 mini-batches
            resnet50.eval()
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Train acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0
            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        resnet50.train()
print('Training is finished!')

Epoch [1/1]. Step [101/313]. Loss: 0.031. Train acc: 0.129. Test acc: 0.276
Epoch [1/1]. Step [201/313]. Loss: 0.025. Train acc: 0.254. Test acc: 0.374
Epoch [1/1]. Step [301/313]. Loss: 0.023. Train acc: 0.300. Test acc: 0.421
Training is finished!
